# Building Interactive Visualizations in Jupyter Notebook


## What's Jupyter Notebook?
Jupyter Notebook (formerly IPython notebook) is a web application for scientific computation. jupyter supports over 40 languages (called kernals) including R and Matlab. Jupyter is growing in popularity and has become a standard tool for data scientists and researchers.

Python has a wealth of really fantastic libraries for analyzing and plotting data, including matplotlib, numpy and pandas. However, the plots are generated as static images. 

**Wouldn't it be more interesting have interactive plots?**


## Bokeh
For this demonstration, we'll use the [Bokeh](http://bokeh.pydata.org/en/latest/). We first need to tell  Bokeh to load its JavaScript component, BokehJS, using the ```output_notebook()``` function.

In [9]:
from bokeh.plotting import output_notebook, show

output_notebook()

Loading BokehJS ...

## Sample Data
A samll set of data, ```autompg```, available in the ```bokeh.sampledata``` package. ```autompg``` is in the ```pandas.dataframe``` format which you can think of as a database table or spreadsheet.

In [2]:
from bokeh.sampledata.autompg import autompg as df

df

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl


### Charts
Bokeh provides high level statistical plots such as bar charts, horizon plots and time series using the ```bokeh.charts``` interface. 

The interface is geared to be extremely simple to use in conjunction with the [Pandas](http://pandas.pydata.org/) data structures library (```Series``` or ```DataFrame```).

In the first plot below, we use the ```Bar``` chart to show the average miles per gallon for a given cylinder type. Note that we specify the ```mean``` aggregation type (default is sum).

In [19]:
from bokeh.charts import Bar

barchart = Bar(df, label='cyl', values="mpg", agg='mean', title="Average MPG By Cycle")
show(barchart)

In [21]:
from bokeh.charts import Scatter

scatter = Scatter(df, x='mpg', y='hp', color='cyl', marker='origin',
                  title="Auto MPG", xlabel="Miles Per Gallon",
                  ylabel="Horsepower")

show(scatter)

## Bokeh Widget
Widgets provide User Interface (UI) components such as sliders and buttons, making your bokeh visualizations dynamic

## Plotly
[Plotly](https://plot.ly/)  is an enterprise application providing advanced graphing and data analysis within the browser. The python library and associated javascript is open source. By default the service uses the ```plot.ly``` account and servers  to host data but you can also run without a server or with a paid enterprise service on your corporate network.